In [ ]:
import os
import sys
import pandas as pd
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, clear_output
sys.path.insert(0, '/Users/huripari/Documents/PhD/TCs_Genesis/xai-gpi')
import utils_results as ut

In [ ]:
# Define widgets
widget_layout1 = widgets.Layout(width='220px')
widget_layout2 = widgets.Layout(width='170px')

basin_widget = widgets.Dropdown(
    options=['NA', 'NEP', 'NWP', 'NI', 'SP', 'SI'],
    value='NEP',
    style={'description_width': 'initial'},
    layout=widget_layout1,
)
label_basin_widget = widgets.HTML(value="<b>Basin:</b>", layout=widget_layout2)
basin_box = HBox([label_basin_widget, basin_widget])

update_button = widgets.Button(
    description = 'Update',
    icon = "check",
    layout=widgets.Layout(height='auto'),
    style={'button_color': 'lightgreen'}
)

In [ ]:
def update_output(basin):
    # Load the data containing the performances of the models
    project_dir = '/Users/huripari/Documents/PhD/TCs_Genesis'
    fs_dir = os.path.join(project_dir, 'xai-gpi')
    results_dir = os.path.join(fs_dir, 'results')
    figures_dir = os.path.join(fs_dir, 'best_model_analysis', 'figures')
    basin_fig_dir = os.path.join(figures_dir, basin)
    cluster_data_info = {'GLB': 'nc12', 'NEP': 'DSnc12', 'NWP': 'Anc10', 'NA': 'DSnc6', 'NI': 'DSnc12', 'SI':'DSnc9', 'SP': 'nc7'}
    cluster_data = cluster_data_info[basin]
    ## Annual time series ##
    figY_path = os.path.join(basin_fig_dir, f'annual_time_series_{basin}.pdf')
    figY_noT_path = os.path.join(basin_fig_dir, f'annual_time_series_noT_{basin}.pdf')
    figY_img = ut.load_pdf_convert_to_image(figY_path)
    figY_noT_img = ut.load_pdf_convert_to_image(figY_noT_path)
    figy_box = HBox([
        VBox([
            widgets.HTML(value='<div style="text-align: center;"><h2>Detrended Annual time series</h2></div>'),
            ut.PIL_to_widget(figY_noT_img, 1200, 600),
              ]),
        VBox([
            widgets.HTML(value='<div style="text-align: center;"><h2>Annual time series</h2></div>'),
            ut.PIL_to_widget(figY_img, 1200, 600),
              ]),
    ])
    display(figy_box)
    ## Selected variables ##
    sel_feat_path = os.path.join(results_dir, f'selected_features_best_models_{basin}_{cluster_data}.csv')
    df_sel_feat_perc = pd.read_csv(sel_feat_path, index_col=0)
    selected_features = df_sel_feat_perc['80'].dropna().to_list()
    fig_cluster_path = os.path.join(basin_fig_dir, f'cluster_var_map_{basin}_{cluster_data}_perc80.pdf')
    fig_cluster = ut.load_pdf_convert_to_image(fig_cluster_path)
    sel_features_widget = VBox([widgets.Label(text) for text in selected_features])
    sel_feature_box = VBox([
        widgets.HTML(value='<div style="text-align: center;"><h2>Selected features</h2></div>'),
        HBox([sel_features_widget, ut.PIL_to_widget(fig_cluster, 1200, 800)]),
    ])
    display(sel_feature_box)
    ## SHAP values ##
    fig_shap_path = os.path.join(basin_fig_dir, f'shap_analysis_{basin}.pdf')
    fig_shap_maxmin_path = os.path.join(basin_fig_dir, f'shap_analysis_{basin}_years_maxmin.pdf')
    fig_shap = ut.load_pdf_convert_to_image(fig_shap_path)
    fig_shap_maxmin = ut.load_pdf_convert_to_image(fig_shap_maxmin_path)
    shap_box = HBox([
        VBox([
            widgets.HTML(value='<div style="text-align: center;"><h2>SHAP values</h2></div>'),
            ut.PIL_to_widget(fig_shap, 1200, 800),
              ]),
        VBox([
            widgets.HTML(value='<div style="text-align: center;"><h2>SHAP values max/min</h2></div>'),
            ut.PIL_to_widget(fig_shap_maxmin, 1200, 800),
              ]),
    ])
    display(shap_box)

# Binding the widgets together
widgets_layout = HBox([basin_box, update_button])
display(widgets_layout)

outputs = widgets.Output()

def on_button_clicked(b):
    with outputs:
        clear_output(wait=True)
        update_output(basin_widget.value)
update_button.on_click(on_button_clicked)
display(outputs)

Output()